In [ ]:
import numpy as np
import pandas as pd
import eipy.ei as e
import tensorflow as tf
import pickle as pkl
from skimage.transform import resize
from importlib import reload

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.additional_ensembles import MeanAggregation, CES
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

In [ ]:
ensemble_predictors = {
                    'Mean' : MeanAggregation(),
                    'CES' : CES(scoring=lambda y_test, y_pred: fmax_score(y_test, y_pred)[0]),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(),
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

In [ ]:
from keras import layers, models
from keras.applications import ResNet50, MobileNet, InceptionV3, DenseNet121

res_model = ResNet50(weights='imagenet', include_top=False)
for layer in res_model.layers:
    layer.trainable = False
collecting_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

base_model = MobileNet(weights='imagenet', include_top=False)
x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
net_model = models.Model(inputs=base_model.input, outputs=predictions)
net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

pretrained_model = InceptionV3(weights='imagenet', include_top=False)
for layer in pretrained_model.layers:
    layer.trainable = False
collecting_layer = layers.GlobalAveragePooling2D()(pretrained_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
v3_model = models.Model(inputs=pretrained_model.input, outputs=output_layer)
v3_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

dense_model = DenseNet121(weights='imagenet', include_top=False)
for layer in pretrained_model.layers:
    layer.trainable = False
collecting_layer = layers.GlobalAveragePooling2D()(pretrained_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
dense121_model = models.Model(inputs=pretrained_model.input, outputs=output_layer)
dense121_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

base_predictors = {"ResNet" : res_model,
                   "MobileNet" : net_model,
                   "Inception" : v3_model,
                   "DenseNet" : dense121_model
}

In [ ]:
metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
train_size = 1000
X_train, y_train = X_train[:train_size], y_train[:train_size]
X_test, y_test = X_test[train_size:train_size+int(train_size/10)], y_test[train_size:train_size+int(train_size/10)]

In [ ]:
inanimate_labels=[0,1,8,9]
def binarize_array(arr):
    mask = np.isin(arr, inanimate_labels)

    arr = np.where(mask, 0, 1)

    return arr
y_train = binarize_array(y_train)
y_test = binarize_array(y_test)

In [ ]:
new_size = (224, 224)

X_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))
X_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))

for i in range(X_train.shape[0]):
    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')
for i in range(X_test.shape[0]):
    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')

In [ ]:
'''
# sanity check
from sklearn.metrics import accuracy_score
res_model.fit(X_train_resized, y_train, batch_size=20)
y_pred = res_model.predict(X_test_resized)
y_pred_argmax = [y > 0.5 for y in y_pred]
y_pred, y_pred_argmax
accuracy_score(y_pred_argmax, y_test)
'''


In [ ]:
####HI OVER HERE###
reload(e)
###HEYOOOOOO#

In [ ]:
EI = e.EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=2,
                        sampling_strategy="hybrid",
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name="dl",
                        model_building=True,
                        )

In [ ]:
EI.fit_base(X_train_resized, y_train, modality_name="images")

In [ ]:
EI.base_summary["metrics"]

In [ ]:
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

In [ ]:
EI.ensemble_summary["metrics"]

In [ ]:
preferred_ensemble_key = EI.ensemble_summary["metrics"].loc["auc"].idxmax()
X_test_dict = {"images": X_test_resized}
y_pred = EI.predict(X_test_dict, ensemble_model_key=preferred_ensemble_key)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = [y > 0.5 for y in y_pred]
accuracy_score(y_pred, y_test)